In [8]:
import os
import shutil
import pandas as pd
from tensorflow.keras import models, layers, activations, \
    optimizers, utils, losses, initializers, metrics, callbacks


In [ ]:
origem_imagens = "../AnaliseImagens/train"

destino_base = "Imagens"

# Verifica se as colunas esperadas existem no CSV
if 'filename' not in df.columns or 'label' not in df.columns:
    raise ValueError("O CSV deve conter as colunas 'filename' e 'label'.")

for _, row in df.iterrows():
    nome_arquivo = row['filename']  
    classe = row['label']          

    origem = os.path.join(origem_imagens, nome_arquivo)

    destino_pasta = os.path.join(destino_base, classe)
    
    # Criar a pasta se não existir
    os.makedirs(destino_pasta, exist_ok=True)

    destino = os.path.join(destino_pasta, nome_arquivo)

    if os.path.exists(origem):  # Verifica se a imagem existe antes de mover
        shutil.move(origem, destino) 




Organização concluída!


In [23]:
path = 'C:/Users/disrct/Desktop/Machine_Learning/AnaliseImagens/Imagens'
epcohs = 100
batch_size = 32 # Número de amostras por batch
patience = 5 # Número máximo de épocas sem melhora antes do early stopping
learning_rate = 0.001 # Taxa de aprendizado

In [14]:
model = models.Sequential([
    layers.Resizing(56, 56), #muda o tamanho de todas as imagens padronizando elas
    layers.Rescaling(1.0/255),#muda os valores do pixel e deixa entre 0 e 1
    layers.RandomRotation((-0.2, 0.2)), #muda a rotação da imagem, girando ela e vendo de outros angulos

    layers.Conv2D(32, (3, 3), #aplicação dos filtros serão 32 filtros 3x3
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)), #pega o valor maximo de cada quadrado nessa caso 2x2

    layers.Conv2D(64, (4, 4), #2048 @ 24x24 64 filtros x 32 imgs
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()          
    ),
    layers.MaxPooling2D((3, 3)), #2048 @ 8x8

    layers.Flatten(), #pega todos os valores e transforma em um vetor gigante +- 125k

    layers.Dropout(0.2), # a cada época serão dropados 20% dos neuronios

    #Camada de processamento
    layers.Dense(200,
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()  
    ),
                  
    layers.Dense(128,
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()  
    ),

    layers.Dense(64,
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()  
    ),

    layers.Dense(1,
        activation =  'sigmoid',
        kernel_initializer =  initializers.RandomNormal()  
    ),

])

In [15]:
model.compile(
    optimizer = optimizers.Adam(
        learning_rate = learning_rate
    ),
    loss = losses.CategoricalCrossentropy(),
    metrics = [
        metrics.CategoricalAccuracy(),
        metrics.Precision(),
        metrics.Recall()
    ]
)

In [20]:
train = utils.image_dataset_from_directory(
    path,
    validation_split = 0.2,
    subset = "training",
    shuffle = True,
    seed = 42,
    image_size = (224, 224),
    batch_size = batch_size 
)

Found 6499 files belonging to 75 classes.
Using 5200 files for training.


In [21]:
test = utils.image_dataset_from_directory(
    path,
    validation_split = 0.2,
    subset = "validation",
    shuffle = True,
    seed = 42,
    image_size = (224, 224),
    batch_size = batch_size 
)

Found 6499 files belonging to 75 classes.
Using 1299 files for validation.


In [24]:
model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        )
        # callbacks.ModelCheckpoint(
        #     filepath = model_path,
        #     save_weights_only = False,
        #     monitor = 'loss',
        #     mode = 'min',
        #     save_best_only = True
        # )
    ]
)

NameError: name 'epochs' is not defined